# Import Required Libraries
Import NumPy, PyTorch, Matplotlib, and the LNNDetector class from the provided module.

In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import sys

sys.path.append('../..')
from sla.detectors import LNNDetector

window_size = 60
batch_size = 100
n_features = 1

normal_data = []
for i in range(batch_size):
    t = np.linspace(0, 4*np.pi, window_size)
    normal_series = np.sin(t) + 0.1 * np.random.randn(window_size)
    normal_data.append(normal_series)
normal_data = np.array(normal_data)

anomalous_data = []
for i in range(batch_size//4):
    t = np.linspace(0, 4*np.pi, window_size)
    anomaly_series = 1.5*np.sin(1.5*t) + 0.3 * np.random.randn(window_size)
    anomalous_data.append(anomaly_series)
anomalous_data = np.array(anomalous_data)

normal_tensor = torch.tensor(normal_data, dtype=torch.float32)
anomalous_tensor = torch.tensor(anomalous_data, dtype=torch.float32)

detector = LNNDetector(
    n_features=n_features,
    seq_len=window_size,
    latent_dim=8,
    learning_rate=0.001,
    epochs=10,
    threshold_multiplier=2.0
)

train_size = int(0.8 * len(normal_data))
train_data = normal_data[:train_size]
val_data = normal_data[train_size:]

history = detector.fit(train_data, val_data, verbose=True)

plt.figure(figsize=(10, 5))
plt.plot(history['train'], label='Training Loss')
plt.plot(history['val'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

normal_scores = detector.anomaly_score(normal_data)
anomalous_scores = detector.anomaly_score(anomalous_data)

plt.figure(figsize=(12, 6))
plt.hist(normal_scores, bins=30, alpha=0.5, label='Normal Data')
plt.hist(anomalous_scores, bins=30, alpha=0.5, label='Anomalous Data')
plt.axvline(detector.threshold, color='r', linestyle='--', label='Threshold')
plt.xlabel('Anomaly Score')
plt.ylabel('Count')
plt.title('Distribution of Anomaly Scores')
plt.legend()
plt.grid(True)
plt.show()